## Librerias

In [1]:
library(FactoClass)
library(bigrquery)
library(dplyr)
library(knitr)
library(DT)
library(DBI)

Cargando paquete requerido: ade4

Cargando paquete requerido: ggplot2

Cargando paquete requerido: ggrepel

Cargando paquete requerido: xtable

Cargando paquete requerido: scatterplot3d


Adjuntando el paquete: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




## Parametros

In [2]:
# Proyecto de GCP
project <- "co-keralty-costomedico"

# Query para traer la data de diabetes
query <- "SELECT * FROM Diabetes_avicena_survival.diabetes_final"

# Configuracion de la conexion a BigQuery
con <- dbConnect(bigrquery::bigquery(),
                 project = project,
                 dataset = "Diabetes_avicena_survival",
                 location = "us-east1")

# Columnas cualitativas
variables_numericas <- c("year",
                         "month",
                         "edad",
                         "IMC",
                         "HDL",
                         "LDL",
                         "trigliceridos",
                         "perimetro_abdominal"
                         #"time_to_event"
                         )

# Columnas categoricas
variables_factores <- c("genero_paciente",
                        "raza_paciente",
                        "nivel_academico_paciente",
                        "ant_cardio",
                        "med_hipertension",
                        "ant_familiar_dm",
                        "hace_ejercicio",
                        "diabetes")

## Carga de datos

In [3]:
# Traer la data desde BigQuery
data <- dbGetQuery(con, query)

In [4]:
# Realizar cambios en cada una de las columnas de la data
data <- data %>% mutate(across(all_of(variables_factores), as.factor))
data <- data %>% mutate(across(all_of(variables_numericas), as.numeric))

# Vista de los cambios realizados
str(data)

tibble [9,032 × 21] (S3: tbl_df/tbl/data.frame)
 $ numero_identificacion_paciente: chr [1:9032] "26151819" "1007107070" "34341427" "94426911" ...
 $ year                          : num [1:9032] 2022 2022 2024 2024 2024 ...
 $ month                         : num [1:9032] 8 8 1 1 1 5 5 9 9 2 ...
 $ fecha                         : Date[1:9032], format: "2022-08-01" "2022-08-01" ...
 $ edad                          : num [1:9032] 70 19 39 50 56 44 45 54 43 31 ...
 $ peso                          : num [1:9032] 82 56 78 91 56 ...
 $ talla                         : num [1:9032] 1.59 1.61 1.6 1.75 1.55 1.63 1.7 1.8 1.69 1.57 ...
 $ IMC                           : num [1:9032] 32.4 21.6 30.5 29.7 23.3 ...
 $ HDL                           : num [1:9032] 45.3 56 64.5 38 51 45.5 46 51.6 30 33.4 ...
 $ LDL                           : num [1:9032] 158 99 102 151 143 ...
 $ trigliceridos                 : num [1:9032] 238 95 162 211 255 ...
 $ perimetro_abdominal           : num [1:9032] 96 75 89.5 

## Caracterizacion

In [5]:
print("Coeficientes de correlacion entre las variables continuas y Diabetes")
kable(centroids(data[, variables_numericas[3:length(variables_numericas)]],
                data$diabetes)$cr * 100,
      digits = 2)

[1] "Coeficientes de correlacion entre las variables continuas y Diabetes"




| edad|  IMC|  HDL|  LDL| trigliceridos| perimetro_abdominal|
|----:|----:|----:|----:|-------------:|-------------------:|
| 0.04| 0.48| 0.26| 0.16|          0.17|                0.35|

In [6]:
print("Valores test para caracterización de variables cuantitativas y Diabetes")
cluster.carac(data[, variables_numericas[3:length(variables_numericas)]],
              data$diabetes,
              tipo.v = "co",
              v.lim = 2,
              dn = 4,
              dm = 4,
              neg = TRUE)

[1] "Valores test para caracterización de variables cuantitativas y Diabetes"


class: 0
                    Test.Value Class.Mean Frequency Global.Mean
HDL                     4.8910    48.6415      7520     48.3392
LDL                     3.7666   113.8423      7520    113.1242
trigliceridos          -3.8840   152.7419      7520    154.4975
perimetro_abdominal    -5.6086    93.6575      7520     93.9987
IMC                    -6.5664    28.3260      7520     28.4939
------------------------------------------------------------ 
class: 1
                    Test.Value Class.Mean Frequency Global.Mean
IMC                     6.5664    29.3293      1512     28.4939
perimetro_abdominal     5.6086    95.6952      1512     93.9987
trigliceridos           3.8840   163.2290      1512    154.4975
LDL                    -3.7666   109.5524      1512    113.1242
HDL                    -4.8910    46.8357      1512     48.3392

In [7]:
print("Pruebas Test y Chi2 para vars cualitativas y Diabetes")
kable(chisq.carac(data[, variables_factores], data$diabetes, decr = FALSE),
      digits = 8)

[1] "Pruebas Test y Chi2 para vars cualitativas y Diabetes"




|                         |         chi2| dfr|       pval|       tval|       phi2|
|:------------------------|------------:|---:|----------:|----------:|----------:|
|genero_paciente          | 6.599930e+00|   1| 0.01019828|  2.3189720| 0.00073073|
|raza_paciente            | 4.886317e+00|   6| 0.55847453| -0.1471027| 0.00054100|
|nivel_academico_paciente | 2.398332e+01|  12| 0.02044756|  2.0445918| 0.00265537|
|ant_cardio               | 2.993082e+01|   1| 0.00000004|  5.3467493| 0.00331386|
|med_hipertension         | 9.178199e-02|   1| 0.76192383| -0.7125046| 0.00001016|
|ant_familiar_dm          | 3.044634e-01|   1| 0.58109768| -0.2047024| 0.00003371|
|hace_ejercicio           | 1.273206e+00|   3| 0.73550525| -0.6295493| 0.00014097|
|diabetes                 | 9.024827e+03|   1| 0.00000000|        Inf| 0.99920580|

In [8]:
# Valores test para caracterización de diabetes
# según las variables cualitativas
data_prueba <- data.frame(data)
test_values <- cluster.carac(data_prueba[,
                               variables_factores[0:
                                                    (length(variables_factores)
                                                     - 1)]
                             ],
                             data_prueba$diabetes)

In [9]:
print("Valores test para las categorias contra Diabetes")
print(test_values)

[1] "Valores test para las categorias contra Diabetes"


class: 0
                                     Test.Value p.Value Class.Cat Cat.Class
ant_cardio.1                              5.596   0.000      85.4      52.0
nivel_academico_paciente.Profesional      3.233   0.001      86.5      12.4
genero_paciente.Masculino                 2.809   0.005      84.5      39.4
genero_paciente.Femenino                 -2.809   0.005      82.4      60.6
nivel_academico_paciente.Normalista      -3.049   0.002      78.9       6.3
ant_cardio.0                             -5.596   0.000      81.1      48.0
                                     Global Weight
ant_cardio.1                           50.7   4579
nivel_academico_paciente.Profesional   11.9   1079
genero_paciente.Masculino              38.8   3506
genero_paciente.Femenino               61.2   5526
nivel_academico_paciente.Normalista     6.7    602
ant_cardio.0                           49.3   4453
------------------------------------------------------------ 
class: 1
                               